# Filtering Mouse Transcriptome

We remove the following gene from the mouse transcriptome:

  * Mettl26-201  
  
  

In [1]:
import gzip

class FastaEntry:
    '''
    Given a header and a sequence, creates a fasta entry.
    The string representation of the fasta entry consists
    of 50 nts per line.
    Do not change the chunk size (= 50)
    TODO : Consider making header and sequence properties
    Also consider putting more checks here for the input
    '''
    def __init__(self , header , sequence ):
        self.header   = header
        self.sequence = sequence

    def reverse_complement(self):
        complements = {"A" : "T" , "a" : "t" ,
                   "C" : "G" , "c" : "g" ,
                   "G" : "C" , "g" : "c" ,
                   "T" : "A" , "t" : "a" ,
                   "N" : "N" , "n" : "n"}
        result = list()

        for i in range(len(self.sequence) - 1 , -1 , -1 ):
            try:
                result.append(complements[self.sequence[i]])
            except IndexError:
                error_message = "Invalid character (%s) in the fasta sequence with header \n" \
                                "%s"%(self.sequence[i] , self.header)
                raise IOError(error_message)
        self.sequence = "".join(result)


    def __str__(self ):
        chunk_size                  = 50 # Do not change this!
        result_list                 = [ '>' +  self.header ]
        sequence_size               = len(self.sequence)
        number_of_remaining_letters = sequence_size
        number_of_processed_letters = 0

        while number_of_remaining_letters > 0:
            if number_of_remaining_letters <= chunk_size:
                result_list.append(self.sequence[ number_of_processed_letters : ])
                number_of_remaining_letters = 0
                number_of_processed_letters = sequence_size
            else:
                new_number_of_processed_letters = number_of_processed_letters + chunk_size
                result_list.append(self.sequence[ number_of_processed_letters : new_number_of_processed_letters])
                number_of_remaining_letters -= chunk_size
                number_of_processed_letters  = new_number_of_processed_letters

        return("\n".join( result_list ) )

############################################################################################


class FastaFile:
    '''
    This object is used to read fasta files into FastaEntry objects.
    For writing fasta files, we only need FastaEntry objects and using
    their str function, we can convert them to string and write to files.
    Note that it can be used as a context manager as well.
    '''

    def __init__(self , file):
        myopen = open
        if file.endswith(".gz"):
            myopen = gzip.open

        if(file):
            self.f = myopen(file , "rt")
        else:
            self.f = stdin

        self.current_header = ""
        self.current_sequence = list()

    #####################################################

    def __enter__(self):
        return self

    #####################################################

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass


    ######################################################

    def __getitem__(self, index):

        for raw_line in self.f:
            line = raw_line.strip()
            if not line:
                continue

            if line[0] == ">":
                if not self.current_header:
                    self.current_header = (line[1:].split())[0]
                    self.current_sequence = list()
                else:
                    this_entry = FastaEntry(header = self.current_header , sequence = "".join(self.current_sequence) )
                    self.current_header = (line[1:].split())[0]
                    self.current_sequence = list()
                    return(this_entry)
            else:
                self.current_sequence.append(line)

        # this returns the last entry
        if len(self.current_sequence) > 0:
            this_entry = FastaEntry(header = self.current_header , sequence = "".join(self.current_sequence) )
            self.current_sequence = list()
            return(this_entry)

        raise IndexError

    #########################################################

    def __del__(self):
        self.f.close()

In [2]:
genes_to_be_removed = [\
                       "Mettl26-201"
                      ]

## Fasta File

In [3]:
input_fasta_path  = "./transcriptome_old/variant_masked_mouse_transcriptome.fa.gz"
output_fasta_path = "./transcriptome/varnt_masked_and_filtered_mouse_transcriptome.fa.gz"

input_fasta = FastaFile(input_fasta_path)

for e in input_fasta:
    header_contents = e.header.split("|")
    
    if header_contents[4].find(genes_to_be_removed[0]) >= 0:
        print(e.header)

ENSMUST00000026827.15|ENSMUSG00000025731.16|OTTMUSG00000035873.4|OTTMUST00000091995.2|Mettl26-201|Mettl26|1243|UTR5:1-456|CDS:457-1071|UTR3:1072-1243|


In [4]:
input_fasta = FastaFile(input_fasta_path)

with gzip.open(output_fasta_path, "wt") as output_stream:
    for e in input_fasta:
        header_contents = e.header.split("|")

        if header_contents[4].find( genes_to_be_removed[0] ) < 0:
            print(e, file = output_stream)

Now, let's make sure that the output does not contain the transcript Mettl26-201:

In [5]:
output_fasta = FastaFile(output_fasta_path)

for e in output_fasta:
    header_contents = e.header.split("|")
    
    if header_contents[4].find(genes_to_be_removed[0]) >= 0:
        print(e.header)

## Annotation File

In [6]:
old_annotation_file = "./transcriptome_old/appris_mouse_v2_actual_regions.bed"
new_annotation_file = "./transcriptome/appris_mouse_v2_filtered_regions.bed"

with open(old_annotation_file, "rt") as input_stream,\
     open(new_annotation_file, "wt") as output_stream:
    for this_line in input_stream:
        contents = this_line.split()
        
        if len(contents) < 5:
            continue
        
        if contents[0].find(genes_to_be_removed[0]) < 0:
            print(this_line, file=output_stream, end="")

## Lengths File

In [7]:
old_lengths_file = "./transcriptome_old/appris_mouse_v2_transcript_lengths.tsv"
new_lengths_file = "./transcriptome/appris_mouse_v2_filtered_transcript_lengths.tsv"


with open(old_lengths_file, "rt") as input_stream,\
     open(new_lengths_file, "wt") as output_stream:
    for this_line in input_stream:
        contents = this_line.split()
        
        if len(contents) < 2:
            continue
        
        if contents[0].find(genes_to_be_removed[0]) < 0:
            print(this_line, file=output_stream, end="")

In [8]:
! grep Mettl26-201 ./transcriptome/appris_mouse_v2_filtered_transcript_lengths.tsv

In [9]:
! grep Gapdh ./transcriptome/appris_mouse_v2_filtered_transcript_lengths.tsv

ENSMUST00000118875.7|ENSMUSG00000057666.18|OTTMUSG00000027118.9|OTTMUST00000067084.3|Gapdh-203|Gapdh|1420|UTR5:1-240|CDS:241-1242|UTR3:1243-1420|	1420
ENSMUST00000074758.5|ENSMUSG00000061099.12|OTTMUSG00000043090.3|OTTMUST00000113132.2|Gapdhs-201|Gapdhs|1368|UTR5:1-51|CDS:52-1368|	1368
